In [ ]:
%matplotlib tk
from problematic import Indexer, Projector
from problematic import serialED

In [ ]:
ed = serialED.load("data/*.h5")

In [ ]:
ed.plot(vmax=300)

In [ ]:
beam_center_sigma = 10       # sigma of the gaussian kernel
centers = ed.get_direct_beam_position(sigma=beam_center_sigma)

In [ ]:
background_footprint = 19    # window for the median filter
processed = ed.remove_background(footprint=background_footprint)

In [ ]:
stretch_azimuth = -6.61      # orientation of the major axis of the ellipse
stretch_amplitude = 2.43     # percent difference between the major/minor axes
processed = processed.apply_stretch_correction(azimuth=stretch_azimuth, amplitude=stretch_amplitude, centers=centers)

In [ ]:
processed.find_peaks_interactive(imshow_kwargs={"vmax":300, "cmap":"gray"})

In [ ]:
min_sigma=2               # sigma of the minimum gaussian filter
max_sigma=5               # sigma of the maximum gaussian filter
threshold=1               # minimum intensity threshold for a peak
min_size=30               # minimum number of pixels for a peak
processed = processed.find_peaks_and_clean_images(min_sigma=min_sigma, max_sigma=max_sigma, 
                                                  threshold=threshold, min_size=min_size)

In [ ]:
name = "FAU"              # name of the phase
pixelsize = 0.00433       # pixel per Angstrom
dmin, dmax = 1.0, 10.0    # Angstrom
thickness = 100           # nm used to estimate the width of the reflections (max. excitation error)
params = (24.3450,)       # cell parameters
spgr = "Fd-3m"            # space group
projector = Projector.from_parameters(params, spgr=spgr, name=name, 
                                      dmin=dmin, dmax=dmax, thickness=thickness)
indexer = Indexer.from_projector(projector, pixelsize=pixelsize)

In [ ]:
orientations = processed.find_orientations(indexer, centers)

In [ ]:
refined = processed.refine_orientations(indexer, orientations)
serialED.io_utils.save_orientations(refined)

In [ ]:
ed.orientation_explorer(indexer, refined, imshow_kwargs={"vmax":300, "cmap":"gray"})

In [ ]:
processed.export_indexing_results(fname="orientations.ycsv")

In [ ]:
orientations = serialED.io_utils.load_orientations()
intensities = processed.extract_intensities(orientations=orientations, indexer=indexer)

In [ ]:
m = serialED.serialmerge_intensities(intensities, orientations, n=50)

In [ ]:
processed.save("processed.hdf5")

In [ ]:
processed = serialED.load("processed.hdf5")